In [1]:
# data generator 
# - using the synthetic images to train our network
import os
import numpy as np
from config import config
from fringes import fringe_wrapper
from fringes import Fringes_Generator 
import random
from pathlib import Path
import cv2
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter   
import torch,gc

cfg = config()
cfg.net_dir="./data/train3/"
cfg.model_dir = "./data/model3/"
cfg.tensorboard_dir = "./path/to/log/"
cfg.pattern_size = [1920, 50] #[1920, 50] [512, 1]
cfg.Tp = [30, 33, 36]       #在这个实验中，就先设置成这个周期。
cfg.steps = [7,4,3]
# cfg.gamma = 1.4

#删除文件
def del_files(path_file):
    ls = os.listdir(path_file)
    for i in ls:
        f_path = os.path.join(path_file, i)
        # 判断是否是一个目录,若是,则递归删除
        if os.path.isdir(f_path):
            del_files(f_path)
        else:
            os.remove(f_path)

a = input("是否需要重新生成数据集，请输入Y或者N ：")
if a == 'Y':
    if not os.path.exists(cfg.net_dir):
        os.makedirs(cfg.net_dir)
    del_files(cfg.net_dir)
    # #gamma
    num = input("需要生成的数据集数量")
    for i in range(int(num)):
        cfg.gamma = random.uniform(0.8,2.0)
        # print(cfg.gamma)
        images = fringe_wrapper(cfg, "gamma").generate_all()
        np.savez(f"./data/train3/train_data{i:03d}", image1=images[0], image2=images[1], image3=images[2]) 

    # #harmonic
    # for ind in range(100):
    #     print(ind)
    #     fringe_generator=Fringes_Generator(cfg)
    #     fringe_generator.save_data(ind)



b = input("是否需要重新开始训练，请输入Y或者N ：")
if b == 'Y':
    if not os.path.exists(cfg.model_dir):
        os.makedirs(cfg.model_dir)
    del_files(cfg.model_dir)


In [2]:
# build a MLP model for residual estimation 
#- The linear part is also considered in this network
from torch import nn
from MLP_net import NeuralNetwork

# 定义训练的设备
device = torch.device("cuda") #使用gpu进行训练
gc.collect()
torch.cuda.empty_cache()#清楚cuda缓存

# 定义模型及参数
PSP_Rnet = NeuralNetwork(cfg)
PSP_Rnet = PSP_Rnet.to(device)

# Initialize the learning rate
learning_rate = 2e-2
batch_size = 5

# Initialize the loss function
loss_fn = nn.MSELoss() #均方误差
loss_fn = loss_fn.to(device)

# Initialize the optimizer
optimizer = torch.optim.Adam(PSP_Rnet.parameters(), lr=learning_rate)
# dynamic learning rate
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

self.C :  tensor([[ 0.8019,  0.3569,  0.3569,  0.8019],
        [ 1.0000,  1.4450, -0.8019, -0.8019],
        [-0.8019, -0.8019,  1.4450,  1.0000]], device='cuda:0')


In [3]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import os
import cv2

class CustomImageDataset(Dataset):#自定义图像数据集
    def __init__(self, img_dir, transform=None, target_transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(os.listdir(self.img_dir))

    def __getitem__(self, idx):
        data_name =f"train_data{idx:03d}.npz"
        data_path = os.path.join(self.img_dir, data_name)
        data = np.load(data_path)
        img1_0, img1_3, img1_4 = data["image1"][0], data["image1"][3], data["image1"][4]
        img2_0, img2_1, img2_2 = data["image2"][0], data["image2"][1], data["image2"][2]
        img3_0, img3_1, img3_2 = data["image3"][0], data["image3"][1], data["image3"][2]
        img1_1, img1_2, img1_5, img1_6, img2_3 = data["image1"][1], data["image1"][2], data["image1"][5], data["image1"][6], data["image2"][3]
        inputs = np.stack([img1_0, img1_3, img1_4, img2_0, img2_1, img2_2, img3_0, img3_1, img3_2]).astype(np.float32)
        outputs = np.stack([img1_1, img1_2, img1_5, img1_6]).astype(np.float32)


        # img11_0, img11_3, img11_4 = data["image4"][0], data["image4"][3], data["image4"][4]
        # img22_0, img22_1, img22_2 = data["image5"][0], data["image5"][1], data["image5"][2]
        # img33_0, img33_1, img33_2 = data["image6"][0], data["image6"][1], data["image6"][2]

        # inputs1 = np.stack([img11_0, img11_3, img11_4, img22_0, img22_1, img22_2, img33_0, img33_1, img33_2]).astype(np.float32)
#         print(inputs.shape)
#         print(outputs.shape)

        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return inputs,outputs
    
training_data = CustomImageDataset(cfg.net_dir)
inputs, outputs = training_data[0]
# print(inputs.shape)


In [4]:
train_dataloader = DataLoader(training_data, batch_size)
# test_dataloader = DataLoader(test_data, batch_size=64)

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer,epoch):
    cost = []
    size = len(dataloader.dataset)
    # print("size : ",size)
    for data in enumerate(dataloader):#batch, (X, y)
        batch, imgs = data
        X,y = imgs
        X = X.to(device)
        y = y.to(device)
        
        model = model.to(device)
        pred = model(X)          # 前向传播计算预测值
        pred = pred.to(device)
        loss_fn = loss_fn.to(device)
        loss = loss_fn(pred, y)  #真实值与预测值求均方误差
        # Backpropagation
        optimizer.zero_grad()   # 将模型的参数梯度初始化为0
        loss = loss.to(device)
        loss.backward()         # 反向传播计算梯度
        optimizer.step()        # 更新所有参数

        loss = loss.item()
        cost.append(loss)
        # print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        model.save(optimizer,epoch,loss,batch)
    print(optimizer.state_dict()['param_groups'][0]['lr'])
    print(f"Epoch loss average:{np.mean(cost)}")
    return np.mean(cost)
        


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [6]:
if not os.path.exists(cfg.tensorboard_dir):
    os.makedirs(cfg.tensorboard_dir)
del_files(cfg.tensorboard_dir)
writer = SummaryWriter(cfg.tensorboard_dir) #使用tensorboard查看loss

epochs = 200000

start_epoch = PSP_Rnet.load(optimizer)
for t in range(start_epoch,epochs):
    print(f"Epoch {t}\n-------------------------------")
    loss = train_loop(train_dataloader, PSP_Rnet, loss_fn, optimizer,t)

    if optimizer.state_dict()['param_groups'][0]['lr'] == 0.02:
        scheduler.step() # 需要在优化器参数更新之后再动态调整学习率
    
    writer.add_scalar('manner3_gpu_loss',loss, t)


    if loss < 0.5:
        break
#     test_loop(test_dataloader, PSP_Rnet, loss_fn)

print("Done!")

无保存模型，将从头开始训练！
Epoch 0
-------------------------------
0.02
Epoch loss average:53.54123436272144
Epoch 1
-------------------------------
0.02
Epoch loss average:2.957446837425232
Epoch 2
-------------------------------
0.002
Epoch loss average:0.937663272023201
Epoch 3
-------------------------------
0.002
Epoch loss average:0.8603253802657127
Epoch 4
-------------------------------
0.0002
Epoch loss average:0.8114617019891739
Epoch 5
-------------------------------
0.0002
Epoch loss average:0.802739852964878
Epoch 6
-------------------------------
2e-05
Epoch loss average:0.7966319292783737
Epoch 7
-------------------------------
2e-05
Epoch loss average:0.795425725877285
Epoch 8
-------------------------------


OSError: [Errno 22] Invalid argument: './data/model3/best_model'